# Exercício 04 - Matriz View

$$\text{André Luís Mendes Fakhoury} - 4482145$$

$$\text{Prof. Ricardo Marcacini - USP ICMC 2021/1}$$

### Importando as bibliotecas

In [77]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math

### Inicializando janela

In [78]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 800
largura = 800
window = glfw.create_window(largura, altura, "Câmeras - Matriz View", None, None)
glfw.make_context_current(window)

### Inicializando GLSL 

In [79]:
vertex_code = """
        attribute vec3 position;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
        }
        """

fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando GPU Slots para shaders

In [80]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando nosso código-fonte aos slots solicitados

In [81]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [82]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [83]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [84]:
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem do programa

In [85]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Modelando o objeto (Cubo)

In [86]:
vertices = np.zeros(24, [("position", np.float32, 3)])
vertices['position'] = [
    # Face 1 do Cubo (vértices do quadrado)
    (-0.2, -0.2, +0.2),
    (+0.2, -0.2, +0.2),
    (-0.2, +0.2, +0.2),
    (+0.2, +0.2, +0.2),

    # Face 2 do Cubo
    (+0.2, -0.2, +0.2),
    (+0.2, -0.2, -0.2),         
    (+0.2, +0.2, +0.2),
    (+0.2, +0.2, -0.2),
    
    # Face 3 do Cubo
    (+0.2, -0.2, -0.2),
    (-0.2, -0.2, -0.2),            
    (+0.2, +0.2, -0.2),
    (-0.2, +0.2, -0.2),

    # Face 4 do Cubo
    (-0.2, -0.2, -0.2),
    (-0.2, -0.2, +0.2),         
    (-0.2, +0.2, -0.2),
    (-0.2, +0.2, +0.2),

    # Face 5 do Cubo
    (-0.2, -0.2, -0.2),
    (+0.2, -0.2, -0.2),         
    (-0.2, -0.2, +0.2),
    (+0.2, -0.2, +0.2),
    
    # Face 6 do Cubo
    (-0.2, +0.2, +0.2),
    (+0.2, +0.2, +0.2),           
    (-0.2, +0.2, -0.2),
    (+0.2, +0.2, -0.2)]

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [87]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Enviando vértices

In [88]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [89]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [90]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [91]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

###  Vamos pegar a localização da variável color (uniform) do Fragment Shader para que possamos alterá-la em nosso laço da janela!

In [92]:
loc_color = glGetUniformLocation(program, "color")

### Eventos para modificar a posição da câmera.


In [93]:
cameraPos   = glm.vec3(0.0,  0.0,  1.0);
cameraFront = glm.vec3(0.0,  0.0,  0.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);


def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp
    
    speed = 0.05
    if key == 87 and (action==1 or action==2): # W
        cameraPos += speed * (cameraFront - cameraPos)
    
    if key == 83 and (action==1 or action==2): # S
        cameraPos -= speed * (cameraFront - cameraPos)
    
#     if key == 65 and (action==1 or action==2): # A
#         cameraPos = glm.rotate(cameraPos, -theta, glm.vec3(0, 1, 0))
    
#     if key == 68 and (action==1 or action==2): # D
#         cameraPos = glm.rotate(cameraPos, theta, glm.vec3(0, 1, 0))

    
glfw.set_key_callback(window,key_event)
#glfw.set_cursor_pos_callback(window, mouse_event)

### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [94]:
def model():
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade   
#     matrix_transform = glm.rotate(matrix_transform,math.radians(180),glm.vec3(0.0,0.0,0.1))
#     matrix_transform = glm.translate(matrix_transform, glm.vec3(0.0, 0.0, -0.9))  
    matrix_transform = np.array(matrix_transform).T 
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp  # cameraFront = cameraTarget
    mat_view = glm.mat4(1.0)
    mat_view = glm.lookAt(cameraPos, cameraFront, cameraUp);
    mat_view = np.array(mat_view).T
    return mat_view

def projection():
    global altura, largura
    mat_projection = glm.mat4(1.0)
    # perspective parameters: fovy, aspect, near, far
#     mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 100.0)
    mat_projection = np.array(mat_projection).T
    return mat_projection

### Exibindo janela e loop principal

In [95]:
glfw.show_window(window)

glEnable(GL_DEPTH_TEST) ### importante para 3D

import math
raio = 5
ang_inc = 0.01

while not glfw.window_should_close(window):

    glfw.poll_events()
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)    
    
    mat_model = model()
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)    
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection) 
    
    glUniform4f(loc_color, 1, 0, 0, 1.0) ### vermelho
    glDrawArrays(GL_TRIANGLE_STRIP, 0, 4)
    
    glUniform4f(loc_color, 0, 0, 1, 1.0) ### azul
    glDrawArrays(GL_TRIANGLE_STRIP, 4, 4)
    
    glUniform4f(loc_color, 0, 1, 0, 1.0) ### verde
    glDrawArrays(GL_TRIANGLE_STRIP, 8, 4)
    
    glUniform4f(loc_color, 1, 1, 0, 1.0) ### amarela
    glDrawArrays(GL_TRIANGLE_STRIP, 12, 4)
    
    glUniform4f(loc_color, 0.5, 0.5, 0.5, 1.0) ### cinza
    glDrawArrays(GL_TRIANGLE_STRIP, 16, 4)
    
    glUniform4f(loc_color, 0.5, 0, 0, 1.0) ### marrom
    glDrawArrays(GL_TRIANGLE_STRIP, 20, 4)
    
    glfw.swap_buffers(window)

glfw.terminate()